In [1]:
import pandas as pd
import numpy as np
import sys, os , time, pickle
from tabulate import tabulate
from IPython.display import display

methods_list_aux = {'hardMV':'hard-MV', 'softMV': 'soft-MV', 'DS':'D&S', "Raykar": "Raykar", 
                   'OursGlobal': 'Ours Global','OursIndividualK': 'Ours Individual', 'OursIndividualT':'Ours Individual' }

    
def find_match(string,list_des):
    return [value for value in list_des if string in value]

def plot_files(file_name):
    print("\nFile to show: ",file_name)
    table = pd.read_csv(file_name)
    display(table)
    #print(tabulate(table, headers='keys', tablefmt='rst')) #latex
    
def calculate_aux(archivo,what_to_do="mv",model=""):
    try:
        f = open(archivo)
        result_list = []
        for line in f.readlines():
            if what_to_do == "mv":
                if "ACC MV" in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
            elif what_to_do == 'epochs':
                if "Epochs to converge" in line and model in line: 
                    result_list.append(float(line.split(" ")[-1].strip()))
        f.close()
        return result_list
    except:
        return "-"
    
def compare_table(tipo,repre="normal", std=False):
    t = pd.DataFrame()
    t["Methods"] = ["Ann(tr)"]+methods_list
    t["Train"] = ["-"] + ["-" for _ in range(len(methods_list))]
    t["Test"] = ["-"] + ["-" for _ in range(len(methods_list))]
    if tipo == 'acc':
        try:
            t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results.txt",what_to_do="mv"))
        except:
            try:
                t.loc[0,"Train"] = np.mean(calculate_aux(folder+"results_ind.txt",what_to_do="mv"))
            except:
                print("No se encontró acc")
            
    for k, method in enumerate(methods_list):
        aux = find_match(method,files)
        if tipo == "acc" or tipo=="f1": #como normal?
            if tipo=="acc":
                to_extract = "Accuracy"
            elif tipo == "f1":
                to_extract = "F1 (macro)"
            if std:
                test_aux = find_match("_train_std.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])[to_extract][0], 4)
                train_aux = find_match("_test_std.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])[to_extract][0], 4)
            else:
                test_aux = find_match("_train.csv",aux)
                t.loc[k+1,"Train"] = round(pd.read_csv(folder+test_aux[-1])[to_extract][0], 4)
                train_aux = find_match("_test.csv",aux)
                t.loc[k+1,"Test"] = round(pd.read_csv(folder+train_aux[-1])[to_extract][0], 4)

        elif "norm" in tipo.lower() or "js" in tipo.lower():
            if std:
                kl_aux = find_match("_train_std.csv",aux)
            else:
                kl_aux = find_match("_train.csv",aux)
            
            if len(kl_aux) != 0:                
                if repre=="R":
                    to_extract = "("+repre+") "+tipo+ " mean"
                elif repre=='G':
                    to_extract = "("+repre+") "+tipo
                try:
                    t.loc[k+1,"Train"] = round(pd.read_csv(folder+kl_aux[-1])[to_extract][0] ,4)
                except:
                    pass
        elif tipo == "epochs":
            t.loc[k+1,"Train"]=np.mean(calculate_aux(folder+"results.txt",what_to_do="epochs",model=methods_list_aux[method]))
    return t

def get_value(lista, name, std=False):
    i = 0
    if std:
        i = 1
    return np.asarray([value[i][name][0] if type(value[i])== pd.DataFrame else -9999 for value in lista] )

def create_df(method_l,T_try, value, file_re = "./results.txt"):
    t = pd.DataFrame(np.zeros((len(method_l)+1,len(T_try))))    
    t.columns = T_try
    t.index = ["Ann(tr)"]+method_l
    t_std = t.copy()
    
    if value.lower() == "accuracy" or "F1" in value.lower():
        if len(calculate_aux(file_re,what_to_do="mv")) > len(T_try) +1:
            aux2 = calculate_aux(file_re,what_to_do="mv")
            aux = [np.mean(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
            aux2 = [np.std(aux2[i:30*(i+1)]) for i in range(0,len(aux2),30)]
        else:
            aux = calculate_aux(file_re,what_to_do="mv")
            aux2 = np.zeros((len(T_try)))
        t.loc["Ann(tr)",:] = aux
        t_std.loc["Ann(tr)",:] = aux2
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["test"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["test"], value, std=True)

    elif "inertia" in value.lower() or "iner" in value.lower():
        for method_l in methods_list:
            if "Ours" in method_l:
                t.loc[method_l,:] = get_value(dic_res[method_l]["testAux"], value)
                
    else:
        t.index = ["D_t"]+method_l
        t.loc["D_t",:] = D_t
        for method_l in methods_list:
            t.loc[method_l,:] = get_value(dic_res[method_l]["train"], value)
            t_std.loc[method_l,:] = get_value(dic_res[method_l]["train"], value, std=True)
    return t.round(4), t_std.round(4)

!ls

CIFAR_T		Fashion_Q4_0.6	Plot_Results_aux.ipynb	simple_T
Fashion_Q1_0.6	Fashion_Q6_0.5	Plot_Results.ipynb
Fashion_Q3_0.7	LabelMe		Sentiment
Fashion_Q4	LabelMe_v2	Sentiment_Tol3


In [36]:
!ls Fashion_Q4_0.6//
folder = "Fashion_Q4_0.6/"

files = sorted(os.listdir(folder))
methods_list = list(set([f.split("_")[1] for f in os.listdir(folder) if (len(f.split("_")) >2) and ("UpperBound" not in f)]))
methods_list

Fashion_DS_test.csv
Fashion_DS_test_std.csv
Fashion_DS_train.csv
Fashion_DS_train_std.csv
Fashion_hardMV_test.csv
Fashion_hardMV_test_std.csv
Fashion_hardMV_train.csv
Fashion_hardMV_train_std.csv
Fashion_OursGlobal_testAux.csv
Fashion_OursGlobal_testAux_std.csv
Fashion_OursGlobal_test.csv
Fashion_OursGlobal_test_std.csv
Fashion_OursGlobal_train.csv
Fashion_OursGlobal_train_std.csv
Fashion_OursIndividualK_testAux.csv
Fashion_OursIndividualK_testAux_std.csv
Fashion_OursIndividualK_test.csv
Fashion_OursIndividualK_test_std.csv
Fashion_OursIndividualK_train.csv
Fashion_OursIndividualK_train_std.csv
Fashion_OursIndividualT_testAux.csv
Fashion_OursIndividualT_testAux_std.csv
Fashion_OursIndividualT_test.csv
Fashion_OursIndividualT_test_std.csv
Fashion_OursIndividualT_train.csv
Fashion_OursIndividualT_train_std.csv
Fashion_Raykar_test.csv
Fashion_Raykar_test_std.csv
Fashion_Raykar_train.csv
Fashion_Raykar_train_std.csv
Fashion_softMV_test.csv
Fashion_softMV_test_std.csv
Fashion_softMV_train.c

['OursGlobal',
 'Raykar',
 'hardMV',
 'softMV',
 'OursIndividualT',
 'DS',
 'OursIndividualK']

In [30]:
t_acc = compare_table("acc")
t_acc_std = compare_table("acc", std=True)

t_norm_R = compare_table("NormF","R")
t_norm_R_std = compare_table("NormF","R", std=True)
t_js_R = compare_table("JS","R")
t_js_R_std = compare_table("JS","R",  std=True)

t_norm_G = compare_table("NormF","G")
t_norm_G_std = compare_table("NormF","G", std=True)
t_js_G = compare_table("JS","G")
t_js_G_std = compare_table("JS","G", std=True)

t_epochs = compare_table("epochs")

In [21]:
## Q4 -- 0.6 (test es los que fueron eliminados)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

Methods  Train_x  Test_x  Train_y  Test_y
0          Ann(tr)  0.60187       -  0.60187       -
1       OursGlobal   0.6522  0.7745    0.004  0.0085
2           Raykar   0.6438  0.7845   0.0021  0.0051
3           hardMV   0.6011  0.7414   0.0034  0.0045
4           softMV   0.6231  0.7593   0.0034  0.0056
5  OursIndividualT   0.6624  0.7663   0.0026  0.0083
6               DS    0.624  0.7649   0.0041  0.0038
7  OursIndividualK    0.651  0.7526   0.0055  0.0101

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)       -       -       -       -
1       OursGlobal  0.6356  0.7306  0.0072  0.0129
2           Raykar  0.6276  0.7437  0.0069  0.0062
3           hardMV  0.5102  0.6066  0.0084  0.0101
4           softMV  0.6048  0.7092  0.0084  0.0096
5  OursIndividualT  0.6526  0.7338  0.0031  0.0078
6               DS  0.5816  0.6915  0.0113  0.0113
7  OursIndividualK  0.6288  0.7016  0.0131  0.0137

In [24]:
## Q4 -- 0.5 (test es los que fueron eliminados + otros de train)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

Methods   Train_x  Test_x   Train_y  Test_y
0          Ann(tr)  0.686195       -  0.686195       -
1       OursGlobal    0.6777  0.6532    0.0044  0.0055
2           Raykar    0.6551   0.662     0.002  0.0022
3           hardMV    0.6219  0.6221    0.0021  0.0042
4           softMV    0.6402  0.6463    0.0021  0.0045
5  OursIndividualT    0.6979  0.6439     0.003  0.0035
6               DS    0.6364  0.6399    0.0023   0.004
7  OursIndividualK    0.6976   0.638    0.0031  0.0069

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)       -       -       -       -
1       OursGlobal  0.6602  0.6295  0.0079  0.0101
2           Raykar    0.64  0.6436  0.0058  0.0064
3           hardMV  0.5406  0.5329  0.0064  0.0102
4           softMV  0.6213  0.6228  0.0064  0.0094
5  OursIndividualT  0.6867  0.6269  0.0036  0.0028
6               DS  0.5951  0.5942  0.0075  0.0099
7  OursIndividualK  0.6822  0.6166  0.0049  0.0077

In [27]:
## sentiment.. (tol mas alta para mio - 5)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

Methods   Train_x  Test_x   Train_y  Test_y
0          Ann(tr)  0.889578       -  0.889578       -
1       OursGlobal    0.8821  0.7573    0.0038  0.0053
2           Raykar    0.8297  0.7374    0.0124  0.0154
3           hardMV    0.8686  0.7422    0.0167  0.0117
4           softMV    0.7927  0.7443    0.0167  0.0084
5  OursIndividualT    0.8979  0.7634    0.0026  0.0072
6               DS    0.8807  0.7521    0.0435  0.0119
7  OursIndividualK    0.8795  0.7529    0.0055  0.0065

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)       -       -       -       -
1       OursGlobal   0.882  0.7568  0.0039  0.0056
2           Raykar  0.8293  0.7356  0.0127   0.019
3           hardMV  0.8685  0.7413  0.0187  0.0128
4           softMV  0.7915  0.7431  0.0187  0.0099
5  OursIndividualT  0.8979   0.763  0.0026  0.0076
6               DS  0.8802  0.7513  0.0443  0.0131
7  OursIndividualK  0.8793  0.7523  0.0056  0.0069

In [31]:
## sentiment.. Tol 3e-2 (todos)
from IPython.display import display
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))
t_acc = compare_table("f1")
t_acc_std = compare_table("f1", std=True)
display(t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True))

Methods   Train_x  Test_x   Train_y  Test_y
0          Ann(tr)  0.889578       -  0.889578       -
1       OursGlobal    0.8828  0.7547    0.0045  0.0062
2           Raykar     0.831  0.7415    0.0119  0.0081
3           hardMV    0.8382  0.7388    0.0096   0.013
4           softMV    0.7888   0.745    0.0096  0.0073
5  OursIndividualT    0.8998  0.7697    0.0043  0.0053
6               DS    0.8563  0.7447     0.046  0.0122
7  OursIndividualK    0.8851  0.7552    0.0054   0.005

Methods Train_x  Test_x Train_y  Test_y
0          Ann(tr)       -       -       -       -
1       OursGlobal  0.8827  0.7544  0.0046  0.0062
2           Raykar  0.8307    0.74   0.012  0.0092
3           hardMV  0.8377  0.7373  0.0104  0.0149
4           softMV  0.7878  0.7438  0.0104  0.0079
5  OursIndividualT  0.8998  0.7695  0.0043  0.0054
6               DS  0.8558  0.7439  0.0467  0.0131
7  OursIndividualK   0.885  0.7549  0.0055  0.0051

In [ ]:
t_acc.merge(t_acc_std.iloc[:,1:], left_index=True, right_index=True)

In [5]:
from IPython.display import display
display(t_js_R.merge(t_js_R_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_R.merge(t_norm_R_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividualK  0.0789      -  0.0075      -
2           Raykar  0.0285      -  0.0001      -
3               DS  0.0205      -       0      -
4       OursGlobal  0.1045      -  0.0056      -
5           hardMV  0.0601      -       0      -
6           softMV   0.061      -       0      -
7  OursIndividualT  0.0569      -  0.0024      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1  OursIndividualK  0.2004      -  0.0105      -
2           Raykar  0.0909      -  0.0006      -
3               DS  0.0744      -       0      -
4       OursGlobal  0.2278      -  0.0059      -
5           hardMV  0.1788      -       0      -
6           softMV  0.1869      -       0      -
7  OursIndividualT  0.1687      -  0.0047      -

In [16]:
display(t_js_G.merge(t_js_G_std.iloc[:,1:], left_index=True, right_index=True))
display(t_norm_G.merge(t_norm_G_std.iloc[:,1:], left_index=True, right_index=True))

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1       OursGlobal  0.0452      -  0.0042      -
2  OursIndividualK  0.0919      -  0.0078      -
3           Raykar  0.2591      -  0.0006      -
4           softMV  0.0242      -       0      -
5  OursIndividualT  0.0621      -  0.0042      -
6               DS  0.0357      -       0      -
7           hardMV  0.0347      -       0      -

Methods Train_x Test_x Train_y Test_y
0          Ann(tr)       -      -       -      -
1       OursGlobal   0.076      -  0.0038      -
2  OursIndividualK  0.1032      -  0.0062      -
3           Raykar  0.1681      -  0.0003      -
4           softMV  0.0566      -       0      -
5  OursIndividualT  0.0851      -  0.0035      -
6               DS  0.0468      -       0      -
7           hardMV  0.0736      -       0      -

In [22]:
t_epochs

Methods    Train Test
0          Ann(tr)        -    -
1  OursIndividualK  15.0062    -
2           Raykar    4.265    -
3               DS    15.45    -
4       OursGlobal  16.3975    -
5           hardMV    22.65    -
6           softMV     4.15    -
7  OursIndividualT  15.0062    -

## T change

In [5]:
folder = "./simple_T_re//"

In [6]:
to_check = [100,500,1500,3500,6000,10000]

if 'simple' in folder.lower():
    T_data = 5 #for now is this..
    N = 2500
elif "cifar" in folder.lower():
    T_data = 3
    N = 50000
D_t = np.asarray([N*T_data/T for T in to_check])

files_f = [f for f in os.listdir(folder) if ".pickle" in f]
dic_res = {}
for file in files_f:
    #if len(file.split("_")) >2:
    _, method_name, method_type = (file.split(".pickle")[0]).split("_")
    with open(folder+file, 'rb') as handle:
        if method_name in dic_res:
            dic_res[method_name][method_type] = pickle.load(handle)
        else:
            dic_res[method_name] = {}
            dic_res[method_name][method_type] = pickle.load(handle)
methods_list = list(dic_res.keys())
methods_list

['hardMV',
 'OursGlobal',
 'OursIndividualK',
 'Raykar',
 'OursIndividualT',
 'DS',
 'softMV']

In [7]:
t, t_std = create_df(methods_list, to_check,"Accuracy", file_re=folder+'results.txt')
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
Ann(tr)          0.7211  0.7201  0.7197  0.7196  0.7210   0.7229  0.0103   
hardMV           0.7939  0.8046  0.7795  0.7692  0.7825   0.8065  0.0515   
OursGlobal       0.9133  0.9093  0.8941  0.8891  0.9071   0.9122  0.0193   
OursIndividualK  0.9418  0.9459  0.9430  0.9172  0.8775   0.7960  0.0041   
Raykar           0.9405  0.9349  0.9258  0.8435  0.7597   0.7156  0.0076   
OursIndividualT  0.9416  0.9405  0.9171  0.9256  0.9172   0.8479  0.0073   
DS               0.9403  0.9350  0.9289  0.9184  0.8923   0.8598  0.0100   
softMV           0.6825  0.6823  0.6786  0.6619  0.6657   0.6771  0.0344   

                 1500_y  3500_y  6000_y  10000_y  
Ann(tr)          0.0100  0.0098  0.0093   0.0075  
hardMV           0.0689  0.0696  0.0699   0.0480  
OursGlobal       0.1022  0.1010  0.0158   0.0168  
OursIndividualK  0.0080  0.0356  0.0719   0.1216  
Raykar           0.0123  0.1267  0.1223   0.1044  
OursIndividualT  0.0137  0.0271  0.0436   0.0879  
DS               0.0130  0.0202  0.0386   0.0690  
softMV           0.0518  0.0244  0.0307   0.0299

In [8]:
t, t_std = create_df(methods_list,to_check, "(R) JS mean")
#t, t_std = create_df(methods_list,to_check, "(R) NormF mean")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
hardMV           0.0868  0.0856  0.0869  0.0876  0.0870   0.0868  0.0031   
OursGlobal       0.1059  0.1086  0.1144  0.1181  0.1117   0.1114  0.0170   
OursIndividualK  0.0196  0.0208  0.0473  0.0924  0.1389   0.1921  0.0043   
Raykar           0.0144  0.0558  0.1137  0.1569  0.1727   0.1820  0.0011   
OursIndividualT  0.0175  0.0221  0.0541  0.1127  0.1507   0.1927  0.0028   
DS               0.0120  0.0671  0.1771  0.2515  0.2796   0.3049  0.0024   
softMV           0.0871  0.0863  0.0869  0.0875  0.0871   0.0870  0.0027   

                 1500_y  3500_y  6000_y  10000_y  
hardMV           0.0025  0.0015  0.0013   0.0012  
OursGlobal       0.0368  0.0297  0.0156   0.0159  
OursIndividualK  0.0053  0.0053  0.0098   0.0159  
Raykar           0.0015  0.0050  0.0035   0.0031  
OursIndividualT  0.0065  0.0087  0.0077   0.0191  
DS               0.0027  0.0023  0.0033   0.0022  
softMV           0.0017  0.0010  0.0008   0.0010

In [7]:
t, t_std = create_df(methods_list,to_check,"(G) JS")
#t, t_std = create_df(methods_list,to_check,"(G) NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
softMV           0.0139  0.0140  0.0141  0.0137  0.0151   0.0141  0.0000   
OursGlobal       0.0510  0.0475  0.0611  0.0618  0.0653   0.0571  0.0138   
DS               0.0001  0.0025  0.0053  0.0227  0.0401   0.0483  0.0000   
OursIndividualT  0.0036  0.0017  0.0103  0.0046  0.0089   0.0263  0.0006   
hardMV           0.0155  0.0182  0.0143  0.0140  0.0167   0.0176  0.0000   
Raykar           0.0021  0.0218  0.0587  0.0913  0.1076   0.1090  0.0003   
OursIndividualK  0.0061  0.0025  0.0024  0.0086  0.0132   0.0301  0.0014   

                 1500_y  3500_y  6000_y  10000_y  
softMV           0.0000  0.0000  0.0000   0.0000  
OursGlobal       0.0349  0.0146  0.0246   0.0102  
DS               0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0043  0.0006  0.0004   0.0096  
hardMV           0.0000  0.0000  0.0000   0.0000  
Raykar           0.0003  0.0007  0.0024   0.0021  
OursIndividualK  0.0016  0.0023  0.0012   0.0100

In [ ]:
print(tabulate(t,headers='keys',tablefmt='latex'))

In [8]:
t, _ = create_df(methods_list,to_check,"Iner JS")
#t, _ = create_df(methods_list,to_check,"Iner NormF")
t.merge(t_std.iloc[:,1:], left_index=True, right_index=True)

100   500_x  1500_x  3500_x  6000_x  10000_x   500_y  \
Ann(tr)          0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
softMV           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
OursGlobal       0.2539  0.2087  0.1349  0.2122  0.2082   0.2062  0.0138   
DS               0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
OursIndividualT  0.3096  0.3609  0.3964  0.6011  0.6713   0.7111  0.0006   
hardMV           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0000   
Raykar           0.0000  0.0000  0.0000  0.0000  0.0000   0.0000  0.0003   
OursIndividualK  0.2976  0.3421  0.3874  0.5060  0.5801   0.6641  0.0014   

                 1500_y  3500_y  6000_y  10000_y  
Ann(tr)          0.0000  0.0000  0.0000   0.0000  
softMV           0.0000  0.0000  0.0000   0.0000  
OursGlobal       0.0349  0.0146  0.0246   0.0102  
DS               0.0000  0.0000  0.0000   0.0000  
OursIndividualT  0.0043  0.0006  0.0004   0.0096  
hardMV           0.0000  0.0000  0.0000   0.0000  
Raykar           0.0003  0.0007  0.0024   0.0021  
OursIndividualK  0.0016  0.0023  0.0012   0.0100